In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam,SGD
import os
import numpy as np
from PIL import Image ,ImageOps 
import matplotlib.pyplot as plt
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
inception_v3 = InceptionV3(include_top=False, weights='imagenet', input_tensor=None, pooling=None)
model = Sequential()
model.add(inception_v3)
print (model.summary())

/home/chavvi/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10831749999450280044
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7468364596
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2321053522577392011
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, None, None, 2048)  21802784  
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________
None


In [3]:
def get_image(imgpath):
    desired_size = 299
    im_pth = imgpath
    im = Image.open(im_pth)
    old_size = im.size  # old_size[0] is in (width, height) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # use thumbnail() or resize() method to resize the input image
    # thumbnail is a in-place operation
    # im.thumbnail(new_size, Image.ANTIALIAS)
    im = im.resize(new_size, Image.ANTIALIAS)
    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))
    read_img = np.array(new_im)
    mask = read_img!=0
    read_img[mask] = 255
    return read_img



In [4]:
def getFeatures(X):
    tans = model.predict(X)
    ans = [np.mean(avgpool, axis=(0,1)) for avgpool in tans]
    return ans

In [1]:
def name_process(name):
    if(name[0]=='_'):
        return True
    elif (name[0]=='.'):
        return True
    return False
directory = 'VideosCrop'
X = []
y = []
for action_folder in os.listdir(directory):
    action_num = re.findall(r'\d+',action_folder)[0]
    if name_process(action_folder):
        continue
    path = directory + '/' + action_folder
    for frame_folder_num, frame_folder in enumerate(os.listdir(path)):
        one_video = []
        if name_process(frame_folder):
            continue
        frame_folder_path = path + '/' + frame_folder
        for frame in os.listdir(frame_folder_path):
            if name_process(frame):
                continue
            img_path = frame_folder_path +'/' + frame
            if(not frame.endswith('.png')):
                continue
            reframe = get_image(img_path) 
            one_video.append(reframe)
            features = np.zeros((60,2048))
        features[:np.array(one_video).shape[0],:] = getFeatures(np.array(one_video)/255)
        print (np.array(features).shape)
        X.append(features)
        y.append([action_num-1]*features.shape[0])
        print(np.array(one_video).shape)
print (np.array(X).shape)

NameError: name 'os' is not defined

In [31]:
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


encoded_y = to_categorical(y,20)
print (encoded_y)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [51]:
def build_model(maxlen):
    input_videos = Input((maxlen,2048), dtype='float32')
    X = LSTM(128, return_sequences=True)(input_videos)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(20)(X)
    X = Activation('softmax')(X)
    
    model = Model(inputs=input_videos, outputs=X)
    
    return model
print np.array(one_class).shape

(20, 60, 2048)


In [53]:

lstm_model = build_model(60)
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(np.array(X), encoded_y, epochs = 10, batch_size = 10, shuffle=True)

Epoch 1/10
20/20 [==============================] - 4s 221ms/step - loss: 0.5368 - acc: 0.7500
Epoch 2/10
20/20 [==============================] - 1s 40ms/step - loss: 0.1855 - acc: 1.0000
Epoch 3/10
20/20 [==============================] - 1s 41ms/step - loss: 0.0029 - acc: 1.0000
Epoch 4/10
20/20 [==============================] - 1s 40ms/step - loss: 1.1712e-04 - acc: 1.0000
Epoch 5/10
20/20 [==============================] - 1s 40ms/step - loss: 1.1142e-04 - acc: 1.0000
Epoch 6/10
20/20 [==============================] - 1s 40ms/step - loss: 3.0704e-05 - acc: 1.0000
Epoch 7/10
20/20 [==============================] - 1s 41ms/step - loss: 4.1268e-05 - acc: 1.0000
Epoch 8/10
20/20 [==============================] - 1s 40ms/step - loss: 1.4472e-05 - acc: 1.0000
Epoch 9/10
20/20 [==============================] - 1s 40ms/step - loss: 1.2031e-05 - acc: 1.0000
Epoch 10/10
20/20 [==============================] - 1s 40ms/step - loss: 1.2568e-05 - acc: 1.0000


In [2]:
print (03)

3
